uncomment if executing in google colab

In [1]:
# ! pip uninstall pyro-ppl
# ! pip install pyro-ppl==1.5.1
# ! unzip fonts.zip 
# ! unzip claptchagen.zip
# ! unzip csis
# ! cp csis.py /usr/local/lib/python3.7/dist-packages/pyro/infer

In [2]:
import random
import string
import os
import torch.nn as nn
import torch
import torch.nn.functional as F
import pyro
import numpy as np
import pyro.optim as optim
import pyro.distributions as dist
import pyro.infer
import pyro.optim

from torch.autograd import Variable
from pyro.infer import SVI, Trace_ELBO, TraceGraph_ELBO
from PIL import Image
from claptchagen.claptcha import Claptcha
from torch.distributions import constraints
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt
print(pyro.__version__)

captcha_folder = 'generated_captchas'
captchaHeight = 32
captchaWidth = 100
captchaMarginX = 4
captchaMarginY = 4
batch_size = 10

char_dict = string.digits
USE_CUDA = True
MAX_N = 5 # maximum number of letters in a captcha 
MIN_N = 3 # minimum number of letters in a captcha
MIN_NOISE = 0.01 # minimum noise
MAX_NOISE = 0.99 # maximum noise
smoke_test = False
num_steps = 200 if not smoke_test else 10
TrainingSample = 10000 if not smoke_test else 100 # number of captchas generated for training

def randomString():
    """
    return a string with <num_char> random letters
    """
    k = random.randint(MIN_N, MAX_N) # sample number of characters
    
    rndLetters = (random.choice(char_dict) for _ in range(k))
    
    pad_spaces = MAX_N - k # pad the string so the captcha is close to center
    space = " " * (pad_spaces // 2)
    
    return space + "".join(rndLetters) + space

def ramdomNoise():
    """
    return a float between MIN_NOISE, MAX_NOISE
    """
    return random.uniform(MIN_NOISE, MAX_NOISE)

def generate_random_captcha(n, save=False):
    """
    generate n random captchas,
    return a list of texts on the captchas
    """
    # Initialize Claptcha object with random text, FreeMono as font, of size
    # 100x30px, using bicubic resampling filter and adding a bit of white noise
    c = Claptcha(randomString, "fonts/FreeSans.ttf", (captchaWidth, captchaHeight), (captchaMarginX, captchaMarginY),
             resample=Image.BILINEAR, noise=0)
    captcha_generated = [ [] for i in range(MAX_N)]
    for i in range(n):
        c.noise = ramdomNoise()
        if save:
            text, _ = c.write(os.path.join(captcha_folder, 'captcha{}.png'.format(i)))
            os.rename(os.path.join(captcha_folder, 'captcha{}.png'.format(i)),os.path.join(captcha_folder, '{}.png'.format(text + "_" + str(i))))
        text, image = c.image
        text = text.strip()
        image = np.array(image)[:, :, 0] # the generator is gray scale, only keep one channel is enough
        captcha_generated[len(text) - 1].append((text, image, c.noise))
    return captcha_generated
    
captcha_generated = generate_random_captcha(TrainingSample, save=False)
for lst in captcha_generated:
    print("number of samples in group", len(lst))
    # print some sample captcha information generated
    for i, t in enumerate(lst):
        print("text", t[0], "captcha shape", t[1].shape, "noise", t[2])
        if i >= 10:
            break

1.5.1
number of samples in group 0
number of samples in group 0
number of samples in group 3371
text 434 captcha shape (32, 100) noise 0.6349354844830783
text 697 captcha shape (32, 100) noise 0.43285027776769824
text 969 captcha shape (32, 100) noise 0.581969791561952
text 169 captcha shape (32, 100) noise 0.5871694142861202
text 290 captcha shape (32, 100) noise 0.08358693206295223
text 052 captcha shape (32, 100) noise 0.2886126061463699
text 075 captcha shape (32, 100) noise 0.09254628232266757
text 079 captcha shape (32, 100) noise 0.9784430823215315
text 740 captcha shape (32, 100) noise 0.4975907364275721
text 916 captcha shape (32, 100) noise 0.3117240310227761
text 047 captcha shape (32, 100) noise 0.1596947528951785
number of samples in group 3326
text 4566 captcha shape (32, 100) noise 0.24769946854952024
text 8343 captcha shape (32, 100) noise 0.7220132521236297
text 8015 captcha shape (32, 100) noise 0.07853307290697525
text 8624 captcha shape (32, 100) noise 0.66296599351

In [3]:
def render_image(chars, fonts="fonts/FreeSans.ttf", size=(captchaWidth, captchaHeight), 
                 margin=(captchaMarginX, captchaMarginY), resample=Image.BILINEAR, noise=0.3, use_cuda=False):
    #noise = noise.data.item()
    #print(chars, noise)
    pad_spaces = MAX_N - len(chars)
    space = " " * (pad_spaces // 2)
    chars = space + chars + space
    render = Claptcha(chars, fonts, size, margin, resample=resample, noise=noise)

    
    _ , rendered_image = render.image
    rendered_image = np.array(rendered_image)[:,:,0] # the generator is gray scale, only keep one channel is enough
    rendered_image = np.subtract(np.divide(rendered_image, 255), 0.5)
    rendered_image = torch.from_numpy(rendered_image)
    if use_cuda:
        rendered_image = rendered_image.cuda()
    return rendered_image

In [4]:
class CaptchaDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, raw_captchas, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.raw_captchas = raw_captchas
        self.transform = transform

    def __len__(self):
        return len(self.raw_captchas)

    def __getitem__(self, idx):
        label = self.raw_captchas[idx][0]
        image = self.raw_captchas[idx][1]
        noise = self.raw_captchas[idx][2]
        
        image = np.subtract(np.divide(image, 255), 0.5)
        image = torch.from_numpy(image).float()

        if self.transform:
            image = self.transform(image)

        return label, image, noise

In [5]:
def make_loarders(BATCH_SIZE, raw_samples):
    dataloaders = [] # dataloaders for different num of char
    for lst in raw_samples:
        if lst:
            ds = CaptchaDataset(lst)
            dataloader = DataLoader(ds, batch_size=BATCH_SIZE,
                                    shuffle=True, num_workers=0, drop_last=True)
            dataloaders.append(dataloader)
    return dataloaders

def make_batches(dataloaders):
    all_batches = []
    for dl in dataloaders:
        for i_batch, sample in enumerate(dl):
            all_batches.append(sample)
    random.shuffle(all_batches)
    random.shuffle(all_batches)
    return all_batches

TrainLoaders = make_loarders(BATCH_SIZE=batch_size, raw_samples=captcha_generated)

In [6]:
class NoiseNet(nn.Module):

    def __init__(self, img_size, out_size = 1):
        """
        Network for learning noise in a captcha
        """
        super(NoiseNet, self).__init__()
        
        self.img_size = img_size
        self.fc0 = nn.Linear(img_size[0] * img_size[1], img_size[0] * img_size[1])
        self.fc1 = nn.Linear(img_size[0] * img_size[1], 1024)
        
        self.fc2 = nn.Linear(1024, 1024)
        self.fc20 = nn.Linear(1024, img_size[0] * img_size[1])
        self.fc21 = nn.Linear(img_size[0] * img_size[1], out_size)
        self.softplus = nn.Softplus()
    
    def forward(self, img):
        BS = img.shape[0]
        img = img.reshape(-1, self.img_size[0] * self.img_size[1])
        hidden = F.relu(self.fc0(img))
        hidden = self.fc1(hidden)
        # mean of noise, used in normal distribution
        noise_map = self.fc20(F.relu(self.fc2(F.relu(hidden))))
        mean =  self.fc21(F.relu(noise_map))
        # std used in normal distribution
        sigma = torch.tensor([[1e-8] for _ in range(BS)]).float()
        if USE_CUDA:
            sigma = sigma.cuda()
        return mean, sigma, noise_map

In [7]:
class NumNet(nn.Module):
    def __init__(self, img_size, out_size = 3):
        """
        Network for learning N, number of letters in a captcha
        """
        super(NumNet, self).__init__()
        self.neural_net = nn.Sequential(
            nn.Linear(img_size[0] * img_size[1], img_size[0] * img_size[1] * 2),
            nn.ReLU(),
            nn.Linear(img_size[0] * img_size[1] * 2, 256),
            nn.ReLU(),
            nn.Linear(256, out_size),
            nn.LogSoftmax(dim=1))
  
    def forward(self, img):
        img = torch.reshape(img, (img.shape[0], img.shape[1] * img.shape[2]))
        prob = self.neural_net(img)
        return prob

In [9]:
class CharNetSingle(nn.Module):
    def __init__(self, img_size, output_size, MAX_N):
        """
        Network for letters in a captcha, given the noise and number of letters
        """
        super(CharNetSingle, self).__init__()
        self.img_size = img_size
        self.n_max = MAX_N
        # observe layers
        self.nnfc = nn.Linear(img_size[0] * img_size[1], img_size[0] * img_size[1])
        self.conv1 = nn.Conv2d(1, 64, 3)
        self.conv2 = nn.Conv2d(64, 64, 3)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.pool = nn.MaxPool2d(2, 2)

        self.pfc1 = nn.Linear(1280 + MAX_N, 2048)
        self.pfc2 = nn.Linear(2048, 1024)
        self.pfc3 = nn.Linear(1024, 1024)
        self.pfc4 = nn.Linear(1024, output_size)
        
        self.convBN1 = nn.BatchNorm2d(64)
        self.convBN2 = nn.BatchNorm2d(64)
        self.convBN3 = nn.BatchNorm2d(64)

    def forward(self, img, i, noise_map):
        
        BATCH_SIZE = img.shape[0]
        # feed the noise_map to a linear layer to tune the values
        noise_map = F.relu(self.nnfc(noise_map))
        noise_map = torch.reshape(noise_map, (BATCH_SIZE, 1, self.img_size[0], self.img_size[1]))
        
        img = torch.reshape(img, (BATCH_SIZE, 1, self.img_size[0], self.img_size[1]))
        # compute the difference between original image and noise map
        # i.e. extract the unnoisy image
        img = torch.sub(img, noise_map)

        img = self.pool(F.relu(self.convBN1(self.conv1(img))))
        img = self.pool(F.relu(self.convBN2(self.conv2(img))))
        img = self.pool(F.relu(self.convBN3(self.conv3(img))))
        
        # flatten
        img = torch.reshape(img, (BATCH_SIZE, 1280))
        
        i = torch.tensor(i)
        if USE_CUDA:
            i = i.cuda()
            
        i = F.one_hot(i, num_classes=self.n_max).float()
        i = i.repeat(BATCH_SIZE, 1)
        
        img = torch.cat((img, i), 1)
        
        # i is the index of character, use different layers for different indicies
        out = F.relu(self.pfc1(img))
        out = F.relu(self.pfc2(out))
        out = F.relu(self.pfc3(out))
        out = self.pfc4(out)
        out = F.log_softmax(out, dim=1)
        
        return out

In [10]:
def inference(t, use_cuda=False):
    """
    one epoch of inference (iterate the training set once)
    """
    loss = 0
    length = TrainingSample
    loss_group = []
    all_batches = make_batches(TrainLoaders)
    for i_batch, sample_batched in enumerate(all_batches):
        
        img = sample_batched[1]
        if use_cuda:
            img = img.cuda()
        IMG = {"captcha" : img}
        imme_loss = csis.step(observations=IMG)
        loss += imme_loss / length

    print("loss at epoch {} is {}".format(t, loss))
    return loss

In [11]:
def test(n = 0, use_train=False, verbose=False, use_cuda=False):
    """
    benchmarking performance on customized or training set
    """
    if use_train:
        TestLoaders = make_loarders(BATCH_SIZE=1, raw_samples=captcha_generated)
    else:
        test_captcha_generated = generate_random_captcha(n, save=False)
        TestLoaders = make_loarders(BATCH_SIZE=1, raw_samples=test_captcha_generated)
    
    total_correct = 0
    char_correct = 0
    total_char = 0
    all_batches = make_batches(TestLoaders)
    noise_difference = 0
    for i_batch, t in enumerate(all_batches):

        label = t[0][0]
        gt_noise = t[2][0]
        img = t[1]

        if use_cuda:
            img = img.cuda()
        
        IMG = {"captcha" : img}
        
        posterior = csis.run(observations=IMG)
        marginal_num = pyro.infer.EmpiricalMarginal(posterior, "num_char")
        marginal_noise = pyro.infer.EmpiricalMarginal(posterior, "noise")
        with torch.no_grad():

            N_index = marginal_num()
            N = N_index + captchaModel.num_char_domain[0]
            noise = captchaModel._map_to_noise_range(marginal_noise()[0])
            sampled_chars = []
            
            # sample characters one by one
            for i in range(N):
                marginal_char = pyro.infer.EmpiricalMarginal(posterior, "char_{}".format(i))()[0]
                if use_cuda:
                    marginal_char.cpu()
                sampled_chars.append(marginal_char)
        
        chars = ""
        for i in range(len(sampled_chars)):
            c = sampled_chars[i]
            chars +=  captchaModel.char_dict[c]
        correct = 0
        
        for p_char, t_char in zip(chars, label):
            if p_char == t_char:
                correct += 1
        noise_difference += abs(float(noise) - float(gt_noise))
        if not verbose:
            print("N_predicted:", int(N), "| Actual N:", len(label), "| Predicted Noise:", round(float(noise), 3), "| Actual Noise:", round(float(gt_noise), 3), "| Predicted Text:", chars, "| Actual Text:", label, "| Correct:", correct)
        if correct == len(label) and int(N) == len(label):
            total_correct += 1
        char_correct += correct
        total_char += len(label)
    num_test_samples = i_batch + 1
    accuracy = total_correct / num_test_samples
    char_accuracy = char_correct / total_char
    noise_difference = noise_difference / num_test_samples
    print("use_train =", use_train, "AVG Noise Difference:", noise_difference, "Total correct:", total_correct, "accuracy:{}/{}=".format(total_correct, num_test_samples), accuracy, "char_accuracy:{}/{}=".format(char_correct, total_char), char_accuracy)


In [12]:
def test_cycle(use_cuda):
    
    # disable dropout
    #captchaModel.numNet.eval()
    #captchaModel.charNetSingle.eval()
    test(use_train=True, verbose=True, use_cuda=use_cuda)
    test(1000, use_train=False, verbose=True, use_cuda=use_cuda)
    #test(10, use_train=True, verbose=False, use_cuda=use_cuda)
    test(10, use_train=False, verbose=False, use_cuda=use_cuda)
    # enable dropout
    #captchaModel.numNet.train()
    #captchaModel.charNetSingle.train()

def optimize(start_epoch=1, use_cuda=False):
    """
    Training/Inferencing Stage
    """
    loss_sequence = []
    pause = 5
    save_pause = 10
    print("Optimizing...")
    for t in range(start_epoch, num_steps + 1):
        L = inference(t, use_cuda)
        loss_sequence.append(L)
        if (t % pause == 0) and (t > 0):
            test_cycle(use_cuda=use_cuda)
        if (t % save_pause == 0) and (t > 0):
            save_and_download_checkpoints("branches-1-no-var-no-tanh_model.pt", "branches-1-no-var-no-tanh_optim.pt", "branches-1-no-var-no-tanh_param_store.pt")
    plt.plot(loss_sequence)
    plt.title("loss")
    plt.show()

In [13]:
# saves the model and optimizer states to disk
def save_checkpoint(currentModel, currentOptimzier, save_model, save_opt, save_param_store):
    print("saving model to %s..." % save_model)
    torch.save(currentModel.state_dict(), save_model)
    print("saving optimizer states to %s..." % save_opt)
    currentOptimzier.save(save_opt)
    print("saving pyro pram store states to %s..." % save_param_store)
    pyro.get_param_store().save(save_param_store)
    print("done saving checkpoints to disk.")

# loads the model and optimizer states from disk
def load_checkpoint(myModel, myOptimzer, load_model, load_opt, load_param_store):
    pyro.clear_param_store()
    print("loading model from %s..." % load_model)
    myModel.load_state_dict(torch.load(load_model))
    print("loading optimizer states from %s..." % load_opt)
    myOptimzer.load(load_opt)
    print("loading pyro pram store states from %s..." % load_param_store)
    pyro.get_param_store().load(load_param_store)
    print("done loading states.")
    pyro.module("guide", myModel, update_module_params=True)

def save_and_download_checkpoints(save_model, save_opt, save_param_store):
    save_checkpoint(captchaModel, optimiser, save_model, save_opt, save_param_store)

In [14]:
class CaptchaModel(nn.Module):
    """
    network, model and guide wrapper class
    """
    def __init__(self, use_cuda=False):
        super().__init__()
        self.num_char_domain = torch.arange(MIN_N, MAX_N + 1)
        if use_cuda:
            self.num_char_domain = self.num_char_domain.cuda()

        self.numNet = NumNet((captchaHeight, captchaWidth), len(self.num_char_domain))
        self.noiseNet = NoiseNet((captchaHeight, captchaWidth), 1)
        self.char_dict = char_dict # letter dictionary
        self.rnn_hidden_size = 512
        self.rnn_num_layer = 2
        self.charNet = CharNetSingle((captchaHeight, captchaWidth), len(self.char_dict), max(self.num_char_domain))
        self.noise_constraint = torch.distributions.constraints.interval(MIN_NOISE, MAX_NOISE)
        if use_cuda:
            self.cuda()
        self.use_cuda = use_cuda
    
    def _map_to_noise_range(self, input):
        """
        map input number to the valid noise range
        """
        input = torch.distributions.transform_to(self.noise_constraint)(input)
        return input

    def guide(self, observations={"captcha": torch.rand(1, captchaHeight, captchaWidth)}):
        pyro.module("guide", self)
        img = observations["captcha"].float()
        
        # posterior to the number of letters
        prob = self.numNet(img)
        prob = torch.mean(prob, dim=0)
        N_index = pyro.sample("num_char", dist.Categorical(prob))
        N_index = torch.add(N_index, self.num_char_domain[0])
        
        with pyro.plate("data", img.shape[0]):
            
            # posterior to the noise
            noise_mean, noise_sig, noise_map = self.noiseNet(img)
            noise_batch = pyro.sample("noise", dist.Normal(noise_mean, noise_sig).to_event(1))
            noise_batch = self._map_to_noise_range(noise_batch)
            
            for i in range(N_index):
                charP_i = self.charNet(img, i, noise_map)
                pyro.sample("char_{}".format(i), dist.Categorical(charP_i).to_event(0))
    
    def model(self, observations={"captcha": torch.rand(1, captchaHeight, captchaWidth)}):
        
        BS = observations["captcha"].shape[0]
        
        num_p = torch.tensor(1 / len(self.num_char_domain)).repeat(len(self.num_char_domain))
        
        if self.use_cuda:
            num_p = num_p.cuda()
        
        # sample the number of characters
        N_index = pyro.sample("num_char", dist.Categorical(num_p))
        N_index = torch.add(N_index,  self.num_char_domain[0])
        
        with pyro.plate("data", BS):
            
            noise_mean = torch.tensor((MAX_NOISE - MIN_NOISE) / 2).repeat((BS, 1))
            noise_sig = torch.tensor(0.5).repeat((BS, 1))

            if self.use_cuda:
                noise_mean = noise_mean.cuda()
                noise_sig = noise_sig.cuda()

            # sample the noise
            noise_batch = pyro.sample("noise", dist.Normal(noise_mean, noise_sig).to_event(1))
            noise_batch = self._map_to_noise_range(noise_batch)
            
            sampled_c = []
            for i in range(int(N_index)):
                num_c_i = torch.tensor(1 / len(self.char_dict)).repeat((BS, len(self.char_dict)))
                if self.use_cuda:
                    num_c_i = num_c_i.cuda()
                c_i = pyro.sample("char_{}".format(i), dist.Categorical(num_c_i).to_event(0))
                sampled_c.append(c_i)
            
            # sample characters
            rendered_images = []
            for i in range(sampled_c[0].shape[0]):
                chars = ""
                for j in range(N_index):
                    chars += self.char_dict[sampled_c[j][i]]
        
                rendered_image = render_image(chars, noise=float(noise_batch[i]), use_cuda=self.use_cuda)
                rendered_images.append(rendered_image)
                
        rendered_images = torch.stack(rendered_images)
        sigma = torch.tensor(0.000001)
        if self.use_cuda:
                sigma = sigma.cuda()

        pyro.sample("captcha", dist.Normal(rendered_images, sigma).to_event(2), obs=observations["captcha"])

captchaModel = CaptchaModel(USE_CUDA)

optimiser = pyro.optim.Adam({'lr': 5e-5})
csis = pyro.infer.CSIS(captchaModel.model, captchaModel.guide, optimiser, num_inference_samples=1)

optimize(1, USE_CUDA)

test_cycle(USE_CUDA)

Optimizing...
loss at epoch 1 is 416062797687710.8
loss at epoch 2 is 116144040722615.31
loss at epoch 3 is 71769594561481.97
loss at epoch 4 is 54744083143668.63
loss at epoch 5 is 47451334466181.54
use_train = True AVG Noise Difference: 0.03664626235574927 Total correct: 238 accuracy:238/10000= 0.0238 char_accuracy:13561/39932= 0.3396023239507162
use_train = False AVG Noise Difference: 0.0364646228731459 Total correct: 26 accuracy:26/1000= 0.026 char_accuracy:1370/4029= 0.3400347480764458
N_predicted: 5 | Actual N: 5 | Predicted Noise: 0.801 | Actual Noise: 0.79 | Predicted Text: 97299 | Actual Text: 68699 | Correct: 2
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.422 | Actual Noise: 0.409 | Predicted Text: 587 | Actual Text: 657 | Correct: 1
N_predicted: 4 | Actual N: 4 | Predicted Noise: 0.306 | Actual Noise: 0.287 | Predicted Text: 4536 | Actual Text: 1546 | Correct: 2
N_predicted: 5 | Actual N: 5 | Predicted Noise: 0.209 | Actual Noise: 0.109 | Predicted Text: 06670 | Actual 

done saving checkpoints to disk.
loss at epoch 21 is 24213826066160.004
loss at epoch 22 is 24021334703498.883
loss at epoch 23 is 23471651951804.414
loss at epoch 24 is 23929980157965.094
loss at epoch 25 is 23310922594110.69
use_train = True AVG Noise Difference: 0.02932792328514663 Total correct: 8955 accuracy:8955/10000= 0.8955 char_accuracy:38829/39932= 0.9723780426725434
use_train = False AVG Noise Difference: 0.029099594904638842 Total correct: 896 accuracy:896/1000= 0.896 char_accuracy:3900/4009= 0.9728111748565728
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.793 | Actual Noise: 0.783 | Predicted Text: 008 | Actual Text: 008 | Correct: 3
N_predicted: 4 | Actual N: 4 | Predicted Noise: 0.42 | Actual Noise: 0.429 | Predicted Text: 5909 | Actual Text: 5909 | Correct: 4
N_predicted: 4 | Actual N: 4 | Predicted Noise: 0.737 | Actual Noise: 0.708 | Predicted Text: 5897 | Actual Text: 5897 | Correct: 4
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.892 | Actual Noise: 0.92 | P

done saving checkpoints to disk.
loss at epoch 41 is 19619659269813.043
loss at epoch 42 is 20794750217460.086
loss at epoch 43 is 20240276969134.49
loss at epoch 44 is 19766181129486.305
loss at epoch 45 is 20475999029703.46
use_train = True AVG Noise Difference: 0.027545802712612248 Total correct: 9446 accuracy:9446/10000= 0.9446 char_accuracy:39361/39932= 0.9857006911749975
use_train = False AVG Noise Difference: 0.027985353701570223 Total correct: 950 accuracy:950/1000= 0.95 char_accuracy:3958/4008= 0.9875249500998003
N_predicted: 5 | Actual N: 5 | Predicted Noise: 0.893 | Actual Noise: 0.928 | Predicted Text: 04394 | Actual Text: 04394 | Correct: 5
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.231 | Actual Noise: 0.216 | Predicted Text: 089 | Actual Text: 089 | Correct: 3
N_predicted: 4 | Actual N: 4 | Predicted Noise: 0.78 | Actual Noise: 0.772 | Predicted Text: 1520 | Actual Text: 1520 | Correct: 4
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.197 | Actual Noise: 0.136 |

done saving checkpoints to disk.
loss at epoch 61 is 18852405711536.117
loss at epoch 62 is 18580661748249.41
loss at epoch 63 is 18872639412856.457
loss at epoch 64 is 18646834335776.75
loss at epoch 65 is 19062425458586.44
use_train = True AVG Noise Difference: 0.025331341791986466 Total correct: 9594 accuracy:9594/10000= 0.9594 char_accuracy:39510/39932= 0.9894320344585796
use_train = False AVG Noise Difference: 0.024360719186757997 Total correct: 966 accuracy:966/1000= 0.966 char_accuracy:3994/4030= 0.9910669975186104
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.407 | Actual Noise: 0.416 | Predicted Text: 755 | Actual Text: 755 | Correct: 3
N_predicted: 4 | Actual N: 4 | Predicted Noise: 0.81 | Actual Noise: 0.82 | Predicted Text: 7753 | Actual Text: 7753 | Correct: 4
N_predicted: 5 | Actual N: 5 | Predicted Noise: 0.78 | Actual Noise: 0.781 | Predicted Text: 95624 | Actual Text: 95622 | Correct: 4
N_predicted: 5 | Actual N: 5 | Predicted Noise: 0.474 | Actual Noise: 0.478 | P

done saving checkpoints to disk.
loss at epoch 81 is 18084072710098.133
loss at epoch 82 is 18177049202956.676
loss at epoch 83 is 17682873700332.324
loss at epoch 84 is 17922930224686.7
loss at epoch 85 is 18052329384699.496
use_train = True AVG Noise Difference: 0.025167938229765225 Total correct: 9626 accuracy:9626/10000= 0.9626 char_accuracy:39545/39932= 0.9903085244916358
use_train = False AVG Noise Difference: 0.02583482243054346 Total correct: 969 accuracy:969/1000= 0.969 char_accuracy:3961/3993= 0.9919859754570498
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.472 | Actual Noise: 0.468 | Predicted Text: 414 | Actual Text: 414 | Correct: 3
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.898 | Actual Noise: 0.969 | Predicted Text: 733 | Actual Text: 733 | Correct: 3
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.721 | Actual Noise: 0.711 | Predicted Text: 887 | Actual Text: 887 | Correct: 3
N_predicted: 3 | Actual N: 3 | Predicted Noise: 0.329 | Actual Noise: 0.314 | Pred

KeyboardInterrupt: 